In [ ]:
%load_ext tensorboard
!kill 275
from datetime import datetime
from packaging import version
import tensorflow
print(tensorflow.__version__)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import time
from numpy import zeros,array

import sys

import pandas as pd

import matplotlib.pyplot as plt

import time

%matplotlib inline

import cv2 as cv

#import tensorflow_addons as tfa

from google.colab import drive

import csv

drive.mount('/content/drive')

path1="/content/drive/My Drive/P-Box-Black-box-assasement/Datasets/Fashion Mnist/"
path2="/content/drive/My Drive/P-Box-Black-box-assasement/Datasets/Mnist/"

%tensorboard --logdir logs

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
root_logdir = os.path.join(os.curdir, "logs\\fit\\")


# importion Ciphered data set for training and test

x_train = np.loadtxt(path2+'One round GCBPM.txt')

x_train = x_train.reshape((60000,28,28))


x_test = np.loadtxt(path2+'One round GCBPMTest.txt')

x_test = x_test.reshape((10000,28,28))

# importion Plain data set for training and test

y_train = np.loadtxt(path+'MnistTrain.txt')

y_train = y_train.reshape((60000,28,28))


y_test = np.loadtxt(path+'MnistTest.txt')

y_test = y_test.reshape((10000,28,28))

x_train.shape


x_train = x_train.reshape(x_train.shape[0],28,28, 1)
x_test = x_test.reshape(x_test.shape[0], 28,28, 1)
y_train = y_train.reshape(y_train.shape[0],28,28, 1)
y_test = y_test.reshape(y_test.shape[0],28,28, 1)
input_shape = (28, 28,1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')




# Importing the required Keras modules containing model and layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

model = keras.Sequential()

model = tf.keras.models.load_model('/content/drive/My Drive/P-Box-Black-box-assasement/Model/')

model.summary()


from keras.callbacks import EarlyStopping
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np

#-----------------------------------------------------------------------------
# Define custom loss functions for regression in Keras 
#-----------------------------------------------------------------------------



# mean squared error (mse) for regression
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def r_square_loss(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - ( 1 - SS_res/(SS_tot + K.epsilon()))
def quantile_loss(q, y, y_p):
        e = y-y_p
        return tf.keras.backend.mean(tf.keras.backend.maximum(q*e, (q-1)*e))        
quantile =0.5



model.compile(optimizer="Adam", loss=lambda y, y_p: quantile_loss(quantile, y, y_p),metrics=[r_square,"mean_squared_error"])

filepath = "/content/drive/My Drive/P-Box-Black-box-assasement/weights/One round GCBPM.hdf5"

metric = 'loss'
EPOCHS=1500
cp_callback =tf.keras.callbacks.ModelCheckpoint(filepath, monitor=metric, verbose=2, save_best_only=True,mode='auto')


start_lr = 0.1
min_lr = 0.00001
max_lr = 0.00005 * 8
#max_lr = 0.00005 * tpu_strategy.num_replicas_in_sync
rampup_epochs = 10
sustain_epochs = 0
exp_decay = .8

def lrfn(epoch):
  if epoch < rampup_epochs:
    return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
  elif epoch < rampup_epochs + sustain_epochs:
    return max_lr
  else:
    return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)

rang = np.arange(EPOCHS)
y = [lrfn(x) for x in rang]
plt.plot(rang, y)
print('Learning rate per epoch:')


training_dataset=(x_train,y_train)
validation_dataset=(x_test,y_test)
train_steps = 60000 // 2000

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
import tensorboard
%reload_ext tensorboard



tensorboard_callback = keras.callbacks.TensorBoard(log_dir=root_logdir,histogram_freq=1)

history =model.fit(x=x_train,y=y_train, validation_data=(x_test,y_test), epochs=EPOCHS,batch_size=2000, verbose=1,steps_per_epoch=train_steps,
                   callbacks=[lr_callback,cp_callback,tensorboard_callback])


import sklearn.metrics, math

from matplotlib import pyplot as plt

def display_training_curves(training, validation, title, subplot):
  ax = plt.subplot(subplot)
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['training', 'validation'])

plt.subplots(figsize=(20,20))
plt.tight_layout()
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)


